In [12]:
import pandas as pd
from openai import OpenAI
from pathlib import Path
import json
import base64

In [13]:
# testing api
system_instructions = {
        "role": "system",
        "content": """You are an assistant that's helping me with this research project. The structure of this project is that we have had undergraduates label traits of images manually. These traits include things like ontology, substrates, font information, text, covid-relation, confidence, and more. You are now going to be doing what they have been doing. When provided with an image, you should return a json object that describes the image according to the structure given here. The structure you should return as is provided below, along with an example. Keep in mind there can be multiple substrates (up to 4), with multiple fonts for each substrate (up to 8). Many of the images will have multiple substrates, look closely in the images for any other substrates which are clear and sharp. For information on what each possible field can mean and examples of each, refer to the files you have available. When annotating the text that is in the image, ensure that the markdown guide that you have available to you is used to record the text.
    
        This is the format you should return to me in, with comments denoting the purpose of the field for some:
    
        |||{return format}
        {
            "substrateCount": // the number of substrates in this image,
            "substrates": [
                {
                    "placement": // refer to placement examples in screenshots of form,
                    "additionalNotes": // not always necessary to include,
                    "thisIsntReallyASign": // set this field to true if this doesn't really fit any of the placement categories and isn't a sign, else false,
                    "notASignDescription": // use this if the previous field was true, describe the placement,
                    "typefaces": [
                        {
                            "typefaceStyle": [], // can be multiple, choose from the options provided to you below,
                            "copy": // make sure that the text is annotated according to the markdown guide in one of the screenshots provided to you,
                            "letteringOntology": [], // refer to the OC Fonts: Codebook Descriptions & Photo Examples file for examples and descriptions of what each of these are,
                            "messageFunction": // again refer to the OC Fonts: Codebook Descriptions & Photo Examples file for examples and descriptions of what each of these are,
                            "covidRelated": // is this text covid related?,
                            "additionalNotes": // any additional notes needed about this image
                        }
                    ],
                    "confidence": // overall confidence in your annotation, 0 being the lowest, 5 the highest,
                    "confidenceReasoning": // reasoning for confidence rating,
                    "additionalInfo": // any additional info about the substrate, not always necessary to include
                }
            ]
        }
        |||
    
        This is an example of the returned product you should give to me. This example only has one substrate but other images may have multiple.
    
        |||{return example}
        {
            "substrateCount": 1,
            "substrates": [
                {
                    "placement": "Window-stuck",
                    "additionalNotes": "decal stickers on a parking meter",
                    "thisIsntReallyASign": false,
                    "notASignDescription": "",
                    "typefaces": [
                        {
                            "typefaceStyle": ["Serif", "Stylized"],
                            "copy": "Please, no food or\\ndrink in the store.\\nThank You!",
                            "letteringOntology": ["Painted", "Pan-face"],
                            "messageFunction": "Operational information",
                            "covidRelated": false,
                            "additionalNotes": "Text is center aligned, \\\"OPEN\\\" is larger than \\\"5PM DAILY\\\""
                        }
                    ],
                    "confidence": 4,
                    "confidenceReasoning": "Could be painted or pan-face or both",
                    "additionalInfo": "Image heavily cut off"
                }
            ]
        }
        |||
    
        These are some of the options for message function, typeface style, ontology and placements:
    
        |||{
            "typeface": [
                "Serif", "Sans serif", "Slab serif", "Script", "Stylized", "Quirky"
            ],
            "lettering_ontology": [
                "Printed", "Decal", "Painted", "Pan channel", "Pan face", "Handmade",
                "Embossed", "Debossed", "Pen or marker", "Reader board", "Spray paint",
                "LED", "Other electronic", "Neon", "Tile", "Chalk", "House number", "Ghost sign"
            ],
            "placements": [
                "Window-stuck", "Window-placed", "Awning/canopy", "Blade", "Fascia",
                "Marquee", "Hanging", "Name-plate", "Painted wall", "Freestanding",
                "Parapet", "Ground", "Bench", "Flag", "Pole-mounted", "Post and panel",
                "Pylon", "Banner", "Wall-placed", "Wall-stuck", "Other-stuck", "Snipe",
                "Graffiti", "Infrastructure", "Memorial", "Sticker"
            ],
            "message_function": [
                "Identification", "Address", "Joint tenant", "Operational information",
                "Advisement/regulation", "Directory", "Generic information", "Menu of options",
                "Commemoration", "Street name", "Advertisement", "Wayfinding", "Infrastructure",
                "Covid-related"
            ]
        }|||
        """
    }

client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)
response


Response(id='resp_680854dc2ccc81919a8e797a7e8124da05851bf7fd48e61b', created_at=1745376476.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_680854de6c088191af8346ed33150ca505851bf7fd48e61b', content=[ResponseOutputText(annotations=[], text='Under a blanket of twinkling stars, a gentle unicorn tiptoed through a silver forest, spreading dreams of sparkling magic to all who slept nearby.', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=18, input_tokens_details=InputTokensDetails(cached_to

In [23]:
import os

template = {
    "custom_id": "",
    "method": "POST",
    "url": "/v1/chat/completions",
    "body": {
        "model": "gpt-4.1",
        "messages": [
            {
                "role": "system",
                "content": system_instructions
            },
            {
                "role": "user",
                "content": "image goes here"
            }
        ],
    }
}
    
def encode_image(images_dir, image_name):
    '''
    Encodes an image into base64 so it can be stored in a jsonl file
    '''

    # find image
    root_path = Path(images_dir)
    for file in root_path.rglob(image_name):  # rglob() searches recursively
        with open(str(file), "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

    print(f"Skipping an image, unable to find {image_name}")
    return None

def create_batch_object(images_path, image_extensions, output_file_name):
    
    # Get all files in the directory (no subfolders)
    image_files = [
        f for f in os.listdir(images_path)
        if os.path.isfile(os.path.join(images_path, f)) and os.path.splitext(f)[1].lower() in image_extensions
    ]
    output_files = []
    
    print("Found", len(image_files), "images in", images_path)
    print("Processing...")
    name_ptr = 0  # ptr for image_files
    i = 0
    while name_ptr < len(image_files):
        with open(f"{output_file_name}{i}.jsonl", "w") as file:
            output_files.append(f"{output_file_name}{i}.jsonl")
            while name_ptr < len(image_files):
                filename = image_files[name_ptr]
                template["custom_id"] = filename.lower()
                base64_image = encode_image(folder_path, filename)
                template["body"]["messages"][1]["content"] = [{
                                    "type": "image_url",
                                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                                }]
                file.write(json.dumps(template) + '\n')
                name_ptr += 1
                if name_ptr % 40 == 0:  # break on every x number of examples
                    break
        i += 1
        
    print("Successfully processed", len(image_files), "images into", i, "separate files")
    return output_files
        
images_path = 'images/Garden Grove'
image_extensions = {'.JPG', '.jpg', '.jpeg'}
output_file_name = "batchGardenGrove" 

output_files = create_batch_object(images_path, image_extensions, output_file_name)

Found 158 images in images/Garden Grove
Processing...
Successfully processed 158 images into 4 separate files


In [24]:
def sendToOpenAI(output_files):
    file_ids = []
    client = OpenAI()
    print("Starting upload of", len(output_files), "files")
    for i in range(len(output_files)):
        filename = output_files[i]
        file_ids.append(client.files.create(
            file=open(filename, "rb"),
            purpose="batch"
        ).id)
        print(f"Finished uploading file {i + 1}")
    print("File ids are", file_ids)
    print("Now starting batch jobs")
    for i in range(len(file_ids)):
        file_id = file_ids[i]
        client.batches.create(
            input_file_id=file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
                "description": "one of many garden grove jobs"
            }
        )
        print(f"Queued batch job {i + 1}")

sendToOpenAI(output_files)

Starting upload of 4 files
Finished uploading file 0
Finished uploading file 1
Finished uploading file 2
Finished uploading file 3
File ids are ['file-1HH2VdGKdgXTZSniSsrL3x', 'file-ML4sC4PgNLAVnfipGz9Kd5', 'file-JJDzCU8CovnpU45tT4Vche', 'file-Hqv4QVaHbMqHDPDffNq6b9']
Now starting batch jobs
Queued batch job 0
Queued batch job 1
Queued batch job 2
Queued batch job 3
